In [1]:
import pandas as pd


In [3]:
branded_food = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18/branded_food.csv")
branded_food.shape

C:\Users\ajinf\AppData\Local\Temp\ipykernel_19572\2523009931.py:1: DtypeWarning: Columns (2,3,4,6,9,12,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  branded_food = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18/branded_food.csv")


(1958978, 20)

In [4]:
import numpy as np
import pandas as pd

def clean_dataframe(df):
    # Identify duplicates
    duplicates = df[df.duplicated()]
    num_duplicates_removed = duplicates.shape[0]

    # Drop duplicates
    df_clean = df.drop_duplicates()

    # Identify columns with more than 50% missing values, excluding those with 'id' in the name
    threshold = len(df_clean) * 0.5
    columns_to_drop = df_clean.columns[(df_clean.isnull().sum() > threshold) & (~df_clean.columns.str.contains('id', case=False))]

    # Drop columns with excessive missing values, but keep 'id' columns
    df_clean = df_clean.drop(columns=columns_to_drop)

    # Fill missing values in 'id' columns with random integers
    id_columns = [col for col in df_clean.columns if 'id' in col.lower()]
    for col in id_columns:
        missing_indices = df_clean[col].isnull()
        random_values = np.random.randint(1000, 9999, size=missing_indices.sum())
        df_clean.loc[missing_indices, col] = random_values

    # Reset index
    df_clean = df_clean.reset_index(drop=True)

    # Print the results
    print(f"Shape before cleaning: {df.shape}")
    print(f"Shape after cleaning: {df_clean.shape}")
    print(f"Number of duplicates found and removed: {num_duplicates_removed}")
    print(f"Columns dropped due to more than 50% missing values: {list(columns_to_drop)} (Total: {len(columns_to_drop)})")
    print("----------")
    missing_values = df_clean.isnull().sum()
    missing_values = missing_values[missing_values > 0]  # Filter for columns with missing values

    if not missing_values.empty:
        print("Columns with missing values and their counts:")
        print(missing_values)
    else:
        print("No missing values found in the cleaned DataFrame.")

    return df_clean


In [7]:
branded_food_cleaned = clean_dataframe(branded_food)

Shape before cleaning: (1958978, 20)
Shape after cleaning: (1958978, 12)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['subbrand_name', 'not_a_significant_source_of', 'household_serving_fulltext', 'package_weight', 'discontinued_date', 'preparation_state_code', 'trade_channel', 'short_description'] (Total: 8)
----------
Columns with missing values and their counts:
brand_owner               17238
brand_name               546806
ingredients                5376
serving_size              10753
serving_size_unit         18993
branded_food_category     10625
modified_date                20
dtype: int64


In [9]:
branded_food_cleaned['brand_owner'] = branded_food_cleaned['brand_owner'].fillna('Unknown')
branded_food_cleaned['brand_name'] = branded_food_cleaned['brand_name'].fillna('Unknown')
branded_food_cleaned['ingredients'] = branded_food_cleaned['ingredients'].fillna('Unknown')
branded_food_cleaned['serving_size_unit'] = branded_food_cleaned['serving_size_unit'].fillna(
    branded_food_cleaned['serving_size_unit'].mode()[0]
)
branded_food_cleaned['branded_food_category'] = branded_food_cleaned['branded_food_category'].fillna('Uncategorized')

branded_food_cleaned['serving_size'] = branded_food_cleaned['serving_size'].fillna(
    branded_food_cleaned['serving_size'].median()
)

branded_food_cleaned['modified_date'] = branded_food_cleaned['modified_date'].fillna(pd.to_datetime('now'))

print("Remaining missing values after imputation:")
print(branded_food_cleaned.isna().sum())

Remaining missing values after imputation:
fdc_id                   0
brand_owner              0
brand_name               0
gtin_upc                 0
ingredients              0
serving_size             0
serving_size_unit        0
branded_food_category    0
data_source              0
modified_date            0
available_date           0
market_country           0
dtype: int64


In [11]:
food_update_log_entry = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\food_update_log_entry.csv")
food_update_log_entry_cleaned = clean_dataframe(food_update_log_entry)
food_update_log_entry_cleaned.head()

Shape before cleaning: (1958978, 3)
Shape after cleaning: (1958978, 3)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
Columns with missing values and their counts:
description    1
dtype: int64


,id,description,last_updated
0,1105904,WESSON Vegetable Oil 1 GAL,2020-11-13
1,1105905,SWANSON BROTH BEEF,2020-11-13
2,1105906,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,2020-11-13
3,1105907,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,2020-11-13
4,1105908,SWANSON BROTH CHICKEN,2020-11-13


In [12]:
food_update_log_entry_cleaned['description'] = food_update_log_entry_cleaned['description'].fillna('Unknown')
food_update_log_entry_cleaned.isna().sum()

id              0
description     0
last_updated    0
dtype: int64

In [15]:
food_attribute_type = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\food_attribute_type.csv")
food_attribute_type_cleaned = clean_dataframe(food_attribute_type)
food_attribute_type_cleaned.head()

Shape before cleaning: (5, 3)
Shape after cleaning: (5, 3)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
No missing values found in the cleaned DataFrame.


,id,name,description
0,998,Update Log,Changes that were made to this food
1,999,Attribute,Generic attributes
2,1000,Common Name,Common names associated with a food.
3,1001,Additional Description,Additional descriptions for the food.
4,1002,Adjustments,"Adjustments made to foods, including moisture ..."


In [17]:
food_attribute = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\food_attribute.csv")
food_attribute_cleaned = clean_dataframe(food_attribute)
food_attribute_cleaned.head()

Shape before cleaning: (2377616, 6)
Shape after cleaning: (2377616, 5)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['seq_num'] (Total: 1)
----------
Columns with missing values and their counts:
name    9487
dtype: int64


,id,fdc_id,food_attribute_type_id,name,value
0,1009437,1105904,4897.0,Ingredients,3
1,1009439,1105905,2792.0,Ingredients,3
2,1009438,1105905,5941.0,Nutrient Removed,6
3,1009440,1105905,1441.0,Nutrient Updated,4
4,1009442,1105906,9466.0,Nutrient Updated,4


In [19]:
food_attribute_cleaned['name'] = food_attribute_cleaned['name'].fillna('Unknown')
food_attribute_cleaned.isna().sum()

id                        0
fdc_id                    0
food_attribute_type_id    0
name                      0
value                     0
dtype: int64

In [21]:
food = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\food.csv")
food_cleaned = clean_dataframe(food)
food_cleaned.head()

C:\Users\ajinf\AppData\Local\Temp\ipykernel_19572\2989601191.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  food = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\food.csv")


Shape before cleaning: (1958978, 8)
Shape after cleaning: (1958978, 7)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['trade_channel'] (Total: 1)
----------
Columns with missing values and their counts:
description    1
dtype: int64


,fdc_id,data_type,description,food_category_id,publication_date,market_country,microbe_data
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,7070.0,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
1,1105905,branded_food,SWANSON BROTH BEEF,9442.0,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
2,1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,8412.0,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
3,1105907,branded_food,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,3419.0,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
4,1105908,branded_food,SWANSON BROTH CHICKEN,2113.0,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."


In [22]:
food_cleaned['description'] = food_cleaned['description'].fillna('Unknown')
food_cleaned.isna().sum()

fdc_id              0
data_type           0
description         0
food_category_id    0
publication_date    0
market_country      0
microbe_data        0
dtype: int64

In [25]:
food_nutrient = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\food_nutrient.csv")
food_nutrient_cleaned = clean_dataframe(food_nutrient)
food_nutrient_cleaned.head()

Shape before cleaning: (25301180, 11)
Shape after cleaning: (25301180, 5)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['data_points', 'min', 'max', 'median', 'footnote', 'min_year_acquired'] (Total: 6)
----------
No missing values found in the cleaned DataFrame.


,id,fdc_id,nutrient_id,amount,derivation_id
0,13706927,1105904,1257,0.00,71.0
1,13706930,1105904,1293,53.33,71.0
2,13706926,1105904,1253,0.00,75.0
3,13706921,1105904,1092,0.00,75.0
4,13706916,1105904,1008,867.00,71.0


In [27]:
microbe = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\microbe.csv")
microbe_cleaned = clean_dataframe(microbe)
microbe_cleaned.head()

Shape before cleaning: (13, 7)
Shape after cleaning: (13, 6)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['max_value'] (Total: 1)
----------
Columns with missing values and their counts:
uom    2
dtype: int64


,id,foodId,method,microbe_code,min_value,uom
0,1,2331974,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,NaN
1,2,2332523,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,NaN
2,3,2336972,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,X_CFG
3,4,2337521,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,X_CFG
4,5,2364024,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,X_CFG


In [29]:
measure_unit = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\measure_unit.csv")
measure_unit_cleaned = clean_dataframe(measure_unit)
measure_unit_cleaned.head()

Shape before cleaning: (122, 2)
Shape after cleaning: (122, 2)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
No missing values found in the cleaned DataFrame.


,id,name
0,1000,cup
1,1001,tablespoon
2,1002,teaspoon
3,1003,liter
4,1004,milliliter


In [31]:
nutrient = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\nutrient.csv")
nutrient_cleaned = clean_dataframe(nutrient)
nutrient_cleaned.head()

Shape before cleaning: (477, 5)
Shape after cleaning: (477, 5)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
Columns with missing values and their counts:
nutrient_nbr    12
rank            11
dtype: int64


,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0


In [33]:
nutrient_cleaned['nutrient_nbr'] = nutrient_cleaned['nutrient_nbr'].fillna(nutrient_cleaned['nutrient_nbr'].mode()[0])
nutrient_cleaned['rank'] = nutrient_cleaned['rank'].fillna(nutrient_cleaned['rank'].mean())
nutrient_cleaned.isna().sum()

id              0
name            0
unit_name       0
nutrient_nbr    0
rank            0
dtype: int64

In [35]:
nutrient_incoming_name = pd.read_csv(r"C:\Users\ajinf\Documents\DS 5110\Project\FoodData_Central_branded_food_csv_2024-04-18\nutrient_incoming_name.csv")
nutrient_incoming_name_cleaned = clean_dataframe(nutrient_incoming_name)
nutrient_incoming_name_cleaned.head()

Shape before cleaning: (730, 3)
Shape after cleaning: (730, 3)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
No missing values found in the cleaned DataFrame.


,id,name,nutrient_id
0,1000,NITROGEN-DUMAS METHO,1002
1,1001,Nitrogen,1002
2,1002,NITROGEN-DUMAS METHOD,1002
3,1003,Nitrogen - Kjeldahl,1002
4,1004,Protein,1003


In [49]:
import os
def save_dataframes_to_csv(dataframes, folder_path):
    os.makedirs(folder_path, exist_ok=True)

    for df_name, df in dataframes.items():
        file_path = os.path.join(folder_path, f"{df_name}.csv")

        df.to_csv(file_path, index=False)
        print(f"Saved {df_name} to {file_path}")

In [51]:
dataframes = {
    'branded_food': branded_food_cleaned,
    'nutrient_incoming_name':nutrient_incoming_name_cleaned,
    'nutrient':nutrient_cleaned,
    'microbe':microbe_cleaned,
    'food_nutrient':food_nutrient_cleaned,
    'food':food_cleaned,
    'food_attribute':food_attribute_cleaned,
    'food_attribute_type':food_attribute_type_cleaned,
    'food_update_log_entry':food_update_log_entry_cleaned,
    'measure_unit':measure_unit_cleaned
}

In [63]:
import pandas as pd
import psycopg2

# Remove the specified columns from the DataFrame
df = branded_food_cleaned

# Convert any necessary columns, e.g., date columns, if required
df['modified_date'] = pd.to_datetime(df['modified_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['available_date'] = pd.to_datetime(df['available_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Database connection details
db_params = {
    'dbname': 'food',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # or your database server address
    'port': '5432'        # default PostgreSQL port
}

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Insert the row into the branded_food table
        cursor.execute("""
            INSERT INTO branded_food (fdc_id, brand_owner, brand_name, gtin_upc, ingredients, 
                serving_size, serving_size_unit, branded_food_category, data_source, modified_date, 
                available_date, market_country)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['fdc_id'],
            row['brand_owner'] if pd.notnull(row['brand_owner']) else None,
            row['brand_name'] if pd.notnull(row['brand_name']) else None,
            row['gtin_upc'] if pd.notnull(row['gtin_upc']) else None,
            row['ingredients'] if pd.notnull(row['ingredients']) else None,
            row['serving_size'] if pd.notnull(row['serving_size']) else None,
            row['serving_size_unit'] if pd.notnull(row['serving_size_unit']) else None,
            row['branded_food_category'] if pd.notnull(row['branded_food_category']) else None,
            row['data_source'] if pd.notnull(row['data_source']) else None,
            row['modified_date'] if pd.notnull(row['modified_date']) else None,
            row['available_date'] if pd.notnull(row['available_date']) else None,
            row['market_country'] if pd.notnull(row['market_country']) else None
        ))
    
    conn.commit()
    print("Data inserted successfully.")
    
except (Exception, psycopg2.DatabaseError) as error:
    print("Error while inserting data:", error)
finally:
    if conn is not None:
        cursor.close()
        conn.close()

Data inserted successfully.


In [45]:
from psycopg2 import sql

df = food_attribute_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, fdc_id, food_attribute_type_id, name, value) VALUES (%s, %s, %s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['fdc_id'], 
                row['food_attribute_type_id'], 
                row['name'], 
                row['value']
            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'food_attribute')


Data inserted successfully


In [77]:
df = food_attribute_type_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, name, description) VALUES (%s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['name'], 
                row['description']
            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'food_attribute_type')

Data inserted successfully


In [59]:
df = food_nutrient_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, fdc_id, nutrient_id, amount, derivation_id) "
                "VALUES (%s, %s, %s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['fdc_id'], 
                row['nutrient_id'], 
                row['amount'],
                row['derivation_id']            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'food_nutrient')

Data inserted successfully


In [91]:
df = food_update_log_entry_cleaned

# Ensure date format is correct
df['last_updated'] = pd.to_datetime(df['last_updated'], errors='coerce')

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, description, last_updated) VALUES (%s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['description'], 
                row['last_updated']            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'food_update_log_entry')

An error occurred: duplicate key value violates unique constraint "food_update_log_entry_pkey"
DETAIL:  Key (id)=(1105904) already exists.



In [93]:
df = measure_unit_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, name) VALUES (%s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['name']
            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'measure_unit')


Data inserted successfully


In [99]:
df = microbe_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, foodId, method, microbe_code, min_value, uom) "
                "VALUES (%s, %s, %s, %s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['foodId'], 
                row['method'], 
                row['microbe_code'], 
                row['min_value'], 
                row['uom']
            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'microbe')



Data inserted successfully


In [101]:
df = nutrient_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, name, unit_name, nutrient_nbr, nutrient_rank) "
                "VALUES (%s, %s, %s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['name'], 
                row['unit_name'], 
                row['nutrient_nbr'], 
                row['rank']
            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'nutrient')

Data inserted successfully


In [107]:
df = nutrient_incoming_name_cleaned

# Function to insert data into PostgreSQL
def insert_data(df, table_name):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Loop through each row in the DataFrame
        for _, row in df.iterrows():
            # Insert each row into the table
            insert_query = sql.SQL(
                "INSERT INTO {table} (id, name, nutrient_id) VALUES (%s, %s, %s)"
            ).format(table=sql.Identifier(table_name))

            cursor.execute(insert_query, (
                row['id'], 
                row['name'], 
                row['nutrient_id']
            ))

        # Commit the transaction
        conn.commit()
        print("Data inserted successfully")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to insert data
insert_data(df, 'nutrient_incoming_name')

Word Counts: {'Hello': 2, 'world': 2, 'from': 1, 'MapReduce': 2, 'of': 1}


In [43]:
import psycopg2


df = food_cleaned
# Connect to PostgreSQL
db_params = {
    'dbname': 'food',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # or your database server address
    'port': '5432'        # default PostgreSQL port
}
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    # Insert each row from the DataFrame into the food table
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO food (fdc_id, data_type, description, food_category_id, publication_date, market_country, microbe_data)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row['fdc_id'],
            row['data_type'],
            row['description'],
            row['food_category_id'],
            row['publication_date'],
            row['market_country'],
            row['microbe_data']
        ))
    
    conn.commit()
    print("Data inserted successfully.")
    
except (Exception, psycopg2.DatabaseError) as error:
    print("Error while inserting data:", error)
finally:
    if conn is not None:
        cursor.close()
        conn.close()

Data inserted successfully.
